In [1]:
import cdsapi
import climetlab as cml
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import geopy
import xarray as xr
import pandas as pd


import csv
import os
import pickle
import os.path

In [2]:
states = ["Alaska", "Alabama", "Arkansas", "Arizona", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia", "Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

In [3]:
c = cdsapi.Client()
c.retrieve(
'reanalysis-era5-single-levels-monthly-means',
{
    'format': 'netcdf',
    'product_type': 'monthly_averaged_reanalysis',
    'variable': [
        '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
        'instantaneous_10m_wind_gust', 'mean_sea_level_pressure', 'surface_pressure',
        'total_precipitation', 'snowfall',
    ],
    'month': [
        '01', '02', '03',
        '04', '05', '06',
        '07', '08', '09',
        '10', '11', '12',
    ],
    'year': [
        '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023',
    ],
    'time': ['00:00', '12:00',]
},
'../../a/state_means/global_mean.nc')


2024-04-11 08:40:54,529 INFO Welcome to the CDS
2024-04-11 08:40:54,530 INFO Sending request to https://cds.climate.copernicus.eu/api/v2/resources/reanalysis-era5-single-levels-monthly-means
2024-04-11 08:40:54,950 INFO Request is queued
2024-04-11 08:40:56,085 INFO Request is running
2024-04-11 08:47:15,517 INFO Request is completed
2024-04-11 08:47:15,519 INFO Downloading https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data8/adaptor.mars.internal-1712843037.3863301-22708-5-57272b6e-f29f-495a-9101-694366e43c5e.nc to ../../a/state_means/global_mean.nc (4.1G)
2024-04-11 10:20:13,485 INFO Download rate 767.8K/s      


Result(content_length=4385538432,content_type=application/x-netcdf,location=https://download-0006-clone.copernicus-climate.eu/cache-compute-0006/cache/data8/adaptor.mars.internal-1712843037.3863301-22708-5-57272b6e-f29f-495a-9101-694366e43c5e.nc)

In [3]:
tablev2 = pd.read_csv("../tables/state_regularized_v2.csv")

In [11]:
print(tablev2)

      Unnamed: 0      Event Type  Valid Date Valid Time  \
0              1       Ice Storm  2002-01-29        NaN   
1              0       Ice Storm  2002-01-30    6:00:00   
2              2       Ice Storm  2002-01-30   16:00:00   
3              3  Severe Weather  2002-03-09    0:00:00   
4              4            Fire  2002-07-20   12:40:00   
...          ...             ...         ...        ...   
1658        1268  Severe Weather  2023-03-31   20:49:00   
1659        1269  Severe Weather  2023-04-01   14:00:00   
1660        1270  Severe Weather  2023-04-29    0:00:00   
1661        1271  Severe Weather  2023-05-01    5:16:00   
1662        1272  Severe Weather  2023-05-22   15:39:00   

                                                  State       Cities  \
0                                                   NaN  Kansas City   
1                                              Oklahoma          NaN   
2                                              Missouri          NaN   
3  

In [6]:
c = cdsapi.Client()
for row in tablev2.iterrows():
    entry = row[1]
    if "nan" not in str(entry['State']) and "DIRTY" not in str(entry['State']):
        date = entry['Valid Date']
        date_split = date.split('-')
        new_date = str(date_split[1]) + "-" + str(date_split[2]) + "-" + str(date_split[0])
        # print(new_date)
        if os.path.isfile("../../a/era5_data/" + new_date + ".nc") == False:
            c.retrieve(
            'reanalysis-era5-single-levels',
            {
                'product_type': 'reanalysis', 'format': 'netcdf', #netcdf
                'year': [
                date_split[0],
                ],
                'month': [
                date_split[1]
                ],
                'day': [
                date_split[2]
                ],
                'time': [
                '00:00', '03:00', '06:00', '09:00',
                '12:00', '15:00', '18:00', '21:00'
                ],
                'variable': [
                '10m_u_component_of_wind', '10m_v_component_of_wind', '2m_temperature',
                'snowfall', 'surface_pressure', 'total_precipitation',
                ],
                },
                '../../a/era5_data/' + new_date + '.nc')
            

